In [2]:
!pip install xgboost

  Using cached xgboost-3.1.1-py3-none-manylinux_2_28_x86_64.whl.metadata (2.1 kB)
  Using cached nvidia_nccl_cu12-2.28.7-py3-none-manylinux_2_18_x86_64.whl.metadata (2.0 kB)
Using cached xgboost-3.1.1-py3-none-manylinux_2_28_x86_64.whl (115.9 MB)
Using cached nvidia_nccl_cu12-2.28.7-py3-none-manylinux_2_18_x86_64.whl (296.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [xgboost]━━━ 1/2 [xgboost]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [21]:
# Load the dataset
df = pd.read_csv('car_fuel_efficiency.csv')

In [22]:
# Display first few rows
df.head()

,engine_displacement,num_cylinders,horsepower,vehicle_weight,acceleration,model_year,origin,fuel_type,drivetrain,num_doors,fuel_efficiency_mpg
0,170,3.0,159.0,3413.433759,17.7,2003,Europe,Gasoline,All-wheel drive,0.0,13.231729
1,130,5.0,97.0,3149.664934,17.8,2007,USA,Gasoline,Front-wheel drive,0.0,13.688217
2,170,NaN,78.0,3079.038997,15.1,2018,Europe,Gasoline,Front-wheel drive,0.0,14.246341
3,220,4.0,NaN,2542.392402,20.2,2009,USA,Diesel,All-wheel drive,2.0,16.912736
4,210,1.0,140.0,3460.870990,14.4,2009,Europe,Gasoline,All-wheel drive,2.0,12.488369


In [23]:
# Select Relevant Columns
selected_columns = [
    'engine_displacement',
    'horsepower',
    'vehicle_weight',
    'model_year',
    'fuel_efficiency_mpg'
]

df = df[selected_columns].copy()

In [24]:
# Check dataset info

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9704 entries, 0 to 9703
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   engine_displacement  9704 non-null   int64  
 1   horsepower           8996 non-null   float64
 2   vehicle_weight       9704 non-null   float64
 3   model_year           9704 non-null   int64  
 4   fuel_efficiency_mpg  9704 non-null   float64
dtypes: float64(3), int64(2)
memory usage: 379.2 KB


In [11]:
# Separate features and target
X = df.drop('fuel_efficiency_mpg', axis=1)
y = df['fuel_efficiency_mpg']

# First split: 60% train, 40% temp
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=1
)

# Second split: 20% validation, 20% test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=1
)

print(f"Train set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

Train set: 5823 samples
Validation set: 1941 samples
Test set: 1941 samples


In [12]:
# Convert to dictionary format
train_dicts = X_train.to_dict(orient='records')
val_dicts = X_val.to_dict(orient='records')
test_dicts = X_test.to_dict(orient='records')

# Initialize and fit DictVectorizer
dv = DictVectorizer(sparse=True)
X_train_vectorized = dv.fit_transform(train_dicts)
X_val_vectorized = dv.transform(val_dicts)
X_test_vectorized = dv.transform(test_dicts)

print(f"Number of features after vectorization: {X_train_vectorized.shape[1]}")
print(f"Feature names example: {dv.get_feature_names_out()[:10]}")

Number of features after vectorization: 6028
Feature names example: ['acceleration=Diesel' 'acceleration=Gasoline' 'acceleration=fuel_type'
 'cylinders=1066.3283007987998' 'cylinders=1223.2982259974249'
 'cylinders=1336.4442307895533' 'cylinders=1414.2414602819395'
 'cylinders=1430.7835669478848' 'cylinders=1443.3823525725488'
 'cylinders=1448.103031837247']


In [14]:
# Train decision tree with max_depth=1
dt = DecisionTreeRegressor(max_depth=1, random_state=1)
dt.fit(X_train_vectorized, y_train)

# Get the feature used for the root split
split_feature_index = dt.tree_.feature[0]
feature_names = dv.get_feature_names_out()
split_feature = feature_names[split_feature_index]

print(f"Feature used for splitting: {split_feature}")

# Evaluate performance
y_pred_val = dt.predict(X_val_vectorized)
rmse_dt = np.sqrt(mean_squared_error(y_val, y_pred_val))
print(f"Decision Tree (max_depth=1) RMSE on validation: {rmse_dt:.3f}")

ValueError: could not convert string to float: 'horsepower'